# Auction Simluation Example

This is an example of how the [Economic Simluation Library (ESL)](https://github.com/EconomicSL) can be integrated into your standard python environment. 

Using only python, we will run an example auction simulation, and then use [pandas](http://pandas.pydata.org/) to analyse the results.

Please note: this will require you to have Scala and Java installed on your computer, more information can be found in the [GitHub repo](https://github.com/EconomicSL/auctions-simulation-example).

In [9]:
import plotly.plotly as py
import cufflinks as cf
import pandas as pd
import collections
import json

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
cf.go_offline()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


## Run the simulation 

The simulation is a scala programme, and the results are written to a local file called `output.json`.

In [6]:
!sbt "run-main ContinuousDoubleAuction"

[warn] Executing in batch mode.
[warn]   For better performance, hit [ENTER] to switch to interactive mode, or
[warn]   consider launching sbt without any commands, or explicitly passing 'shell'
[info] Loading project definition from /Users/ibillett/Projects/auctions-simulation-example/project
[info] Set current project to auctions-simulation-example (in build file:/Users/ibillett/Projects/auctions-simulation-example/)
[info] Running ContinuousDoubleAuction 
[success] Total time: 2 s, completed 04-Jun-2017 19:40:07


## Load the simulation output

The simulation creates a nested JSON file, containing information about each individual trade that was settled during the and filled.

We can load this into pandas and analyse the results we gave generated.

In [11]:
with open('output.json') as simulationOutput:    
    data = json.load(simulationOutput)
    
df = pd.io.json.json_normalize(data)

With the data loaded into pandas, we can take a look at some of the simulation output.

In [12]:
df.head()

,askOrder.issuer,askOrder.limit.value,askOrder.quantity.value,askOrder.tradable.tick,bidOrder.issuer,bidOrder.limit.value,bidOrder.quantity.value,bidOrder.tradable.tick,price.value
0,de4bc3d5-5994-438e-b523-218d63c6cbd8,102948884,1,1,ae6bc827-3145-469f-b17d-36315046a432,676262207,1,1,674661567
1,c44765f9-669a-4f67-9f21-3bee2005dcfb,1227545275,1,1,9200b54f-bb59-4fa0-8a1b-6e940f59db8d,1681268743,1,1,1251951003
2,819b941d-b532-403a-901d-499d88eddcba,326327863,1,1,8fbb1d91-0946-4133-a824-aad130df0fee,676774046,1,1,676518127
3,287b2324-b09c-4490-beb2-4f7ee216bf40,1226299823,1,1,b51910e9-b298-46b5-8044-ea74c6c4a350,1610411243,1,1,1226922549
4,90657a5a-5fef-4849-83ef-97fde57bc822,369835212,1,1,20438c58-7f85-49ea-83ed-7fdc4f9a1083,701566201,1,1,689170124


In [13]:
df.describe()

,askOrder.limit.value,askOrder.quantity.value,askOrder.tradable.tick,bidOrder.limit.value,bidOrder.quantity.value,bidOrder.tradable.tick,price.value
count,3.918000e+03,3918,3918,3.918000e+03,3918,3918,3.918000e+03
mean,8.429955e+08,1,1,1.314118e+09,1,1,1.072899e+09
std,4.866433e+08,0,0,4.859733e+08,0,0,3.386128e+08
min,1.712000e+04,1,1,2.795470e+08,1,1,1.545687e+08
25%,4.139539e+08,1,1,8.930750e+08,1,1,7.856974e+08
50%,8.473279e+08,1,1,1.324776e+09,1,1,1.075602e+09
75%,1.262891e+09,1,1,1.729463e+09,1,1,1.353433e+09
max,1.733015e+09,1,1,2.146966e+09,1,1,1.754388e+09


We can easily calculate the spread of each trade that was settled:

In [69]:
# handling the fact that the stream of auction results is ordered wrong way!
parsed_json = {'ask_price': collections.deque(), 
               'bid_price': collections.deque(), 
               'fill_price': collections.deque(), 
               'spread': collections.deque()}

for fill in fills:
    ask_price = fill['askOrder']['limit']['value']
    bid_price = fill['bidOrder']['limit']['value']
    fill_price = fill['price']['value']
    spread = bid_price - ask_price
    parsed_json['ask_price'].appendleft(ask_price)
    parsed_json['bid_price'].appendleft(bid_price)
    parsed_json['fill_price'].appendleft(fill_price)
    parsed_json['spread'].appendleft(spread)

In [70]:
df = pd.DataFrame.from_dict(parsed_json)

In [71]:
df.head()

,ask_price,bid_price,fill_price,spread
0,53352659,1911251950,154568684,1857899291
1,255784709,1833529520,435358453,1577744811
2,614932197,1399541472,623761534,784609275
3,207850376,329659842,268755109,121809466
4,273324016,1595911716,339069217,1322587700


# Times series plot of price data

Ideally, we should plot the `askOrder.limit.value`, `bidOrder.limit.value`, and the `price.value` on one plot.  Should be the case that the `price.value` is a weighted average of the other two prices.

In [84]:
df[['ask_price', 'bid_price', 'fill_price']].iplot();

In [85]:
df.spread.iplot(logy=True);

In [86]:
returns = df.fill_price.pct_change()
returns.abs().iplot(logy=True);

In [26]:
# TODO: interactive timeseries plot of the data using plotly (or similar)

# Histogram of the distribution of `askOrder.limit.value`

Useful to see how we can create historgram of values across agents.  Agents are representing by `askOrder.issuer` identifiers.

In [88]:
# limit price for ask orders was sampled from U[1, 2147483647]
df.ask_price.iplot(kind='hist')

In [27]:
# TODO: interactive histogram of the data using plotly (or similar)